**Código con el que se han probado otros modelos lexicon como VADER y Textblob. A parte de AFINN**

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install vaderSentiment

In [ ]:
#!pip install afinn

In [ ]:
!pip install --upgrade afinn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53431 sha256=3d713ea4ae53c04b2c163d140ad6ca41b541b99b3613307391f66516dede122a
  Stored in directory: /root/.cache/pip/wheels/ee/d3/a0/f9255ebac29886acb1c28b35b37523f6399677fa06be379f25
Successfully built afinn


In [ ]:
import pkg_resources

version = pkg_resources.get_distribution("afinn").version
print("Versión de AFINN:", version)

Versión de AFINN: 0.2.dev0


In [ ]:
!pip show afinn

Name: afinn
Version: 0.1
Summary: AFINN sentiment analysis
Home-page: https://github.com/fnielsen/afinn
Author: Finn Aarup Nielsen
Author-email: faan@dtu.dk
License: GPL
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [ ]:
import pandas as pd
import string
import re
import nltk
import spacy
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from afinn import Afinn
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from transformers import pipeline, AutoTokenizer

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nlp = spacy.load("en_core_web_sm")

**VADER**

In [ ]:
df_vader=pd.read_csv('translated_reviews.csv')
df_vader.tail()

,listing_id,id,date,reviewer_id,reviewer_name,comments,language,translated_review
1205942,1364616759503109256,1368127111176342942,2025-03-02,368146996,Clémentine,"Très bon séjour chez Juan, le logement était p...",fr,"Very good stay with Juan, the accommodation wa..."
1205943,1365007887657848515,1370244234430465527,2025-03-05,283312107,Hernan Enrique,Recomendable para una persona que quiere hacer...,es,Recommended for a person who wants to do a sho...
1205944,1365007887657848515,1373818429800403177,2025-03-10,184521825,Carlos,"Es una Host súper amable, fue atenta en todo m...",es,"It is a super friendly host, it was attentive ..."
1205945,1365635357360238932,1368103217874537823,2025-03-02,313257078,Yuet,很友善的房东，推荐,zh-cn,NaN
1205946,1366534590465861868,1368063220606082280,2025-03-02,369293146,Hamid,Very friendly host and staff and Great locatio...,en,Very friendly host and staff and Great locatio...


In [ ]:
df_vader.shape

(1205947, 8)

In [ ]:
df_vader.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments',
       'language', 'translated_review'],
      dtype='object')

In [ ]:
df_vader=df_vader.drop(columns=['id', 'date', 'reviewer_id', 'reviewer_name', 'comments',
       'language'])

In [ ]:
df_vader=df_vader.dropna(subset=['translated_review'])

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Quitar URLs
        text = text.strip()  # Quitar espacios
    return text

df_vader["clean_review"] = df_vader["translated_review"].apply(preprocess_text)
df_vader

,listing_id,translated_review,clean_review
0,21853,"My experience in Adel's house was good, althou...","My experience in Adel's house was good, althou..."
1,21853,Adel is a very warm person and thoughtful! He...,Adel is a very warm person and thoughtful! He ...
2,21853,Adel is an exceptionally welcoming host. I was...,Adel is an exceptionally welcoming host. I was...
3,21853,"Adel was very friendly, even the day of my arr...","Adel was very friendly, even the day of my arr..."
4,21853,Abdel has been a magnificent host and the best...,Abdel has been a magnificent host and the best...
...,...,...,...
1205941,1364524253446459367,I loved it. The floor is very new and the loca...,I loved it. The floor is very new and the loca...
1205942,1364616759503109256,"Very good stay with Juan, the accommodation wa...","Very good stay with Juan, the accommodation wa..."
1205943,1365007887657848515,Recommended for a person who wants to do a sho...,Recommended for a person who wants to do a sho...
1205944,1365007887657848515,"It is a super friendly host, it was attentive ...","It is a super friendly host, it was attentive ..."


In [ ]:
analyzer = SentimentIntensityAnalyzer()

df_vader["Sentiment_Score"] = df_vader["clean_review"].astype(str).apply(lambda x: analyzer.polarity_scores(x)["compound"] if isinstance(x, str) else 0)
df_vader

,listing_id,translated_review,clean_review,Sentiment_Score
0,21853,"My experience in Adel's house was good, althou...","My experience in Adel's house was good, althou...",0.7902
1,21853,Adel is a very warm person and thoughtful! He...,Adel is a very warm person and thoughtful! He ...,0.9773
2,21853,Adel is an exceptionally welcoming host. I was...,Adel is an exceptionally welcoming host. I was...,0.7994
3,21853,"Adel was very friendly, even the day of my arr...","Adel was very friendly, even the day of my arr...",0.9753
4,21853,Abdel has been a magnificent host and the best...,Abdel has been a magnificent host and the best...,0.9690
...,...,...,...,...
1205941,1364524253446459367,I loved it. The floor is very new and the loca...,I loved it. The floor is very new and the loca...,0.8559
1205942,1364616759503109256,"Very good stay with Juan, the accommodation wa...","Very good stay with Juan, the accommodation wa...",0.9700
1205943,1365007887657848515,Recommended for a person who wants to do a sho...,Recommended for a person who wants to do a sho...,0.2023
1205944,1365007887657848515,"It is a super friendly host, it was attentive ...","It is a super friendly host, it was attentive ...",0.9115


In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x=df_vader["Sentiment_Score"])

plt.xlabel("Sentiment Score (-1 to 1)")
plt.title("Boxplot de VADER")

plt.show()

In [ ]:
media_sentimiento_VADER = df_vader.groupby('listing_id', as_index=False)['Sentiment_Score'].mean()

In [ ]:
media_sentimiento_VADER

In [ ]:
media_sentimiento_VADER['Sentiment_Score_VADER']=media_sentimiento_VADER['Sentiment_Score']

In [ ]:
media_sentimiento_VADER=media_sentimiento_VADER.drop(columns='Sentiment_Score')

In [ ]:
media_sentimiento_VADER

**Textblob**

In [ ]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import math

In [ ]:
def calculate_sentiment(entry):
    if (type(entry) != str and math.isnan(entry)):
        return -90
    opinion = TextBlob(entry)
    return opinion.sentiment.polarity

In [ ]:
df_textblob=pd.read_csv('translated_reviews.csv')

In [ ]:
df_textblob=df_textblob.dropna(subset=['translated_review'])

In [ ]:
def preprocess_for_textblob(text):
    if isinstance(text, str):
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)
        text = re.sub(r'[^\x00-\x7F]+', '', text)  # Quitar emojis
        text = text.lower()
        text = text.strip()
    return text


df_textblob["clean_review"] = df_textblob["translated_review"].apply(preprocess_for_textblob)

In [ ]:
df_textblob['Sentiment_Score_Textblob'] = df_textblob['clean_review'].apply(calculate_sentiment)
df_textblob

,listing_id,id,date,reviewer_id,reviewer_name,comments,language,translated_review,clean_review,Sentiment_Score_Textblob
0,21853,21051116,2014-10-10,8506071,Pedro Abel,"Mi experiencia en casa de Adel fue buena, aunq...",es,"My experience in Adel's house was good, althou...","my experience in adel's house was good, althou...",0.173333
1,21853,21268157,2014-10-13,19062863,Olga,Adel is a very warm person and thoughtful! He...,en,Adel is a very warm person and thoughtful! He...,adel is a very warm person and thoughtful! he ...,0.473968
2,21853,22527396,2014-11-09,6099461,Hugh,Adel is an exceptionally welcoming host. I was...,en,Adel is an exceptionally welcoming host. I was...,adel is an exceptionally welcoming host. i was...,0.311250
3,21853,22684525,2014-11-11,23357657,Sendy,"Adel fue muy amable, incluso el día de mi lleg...",es,"Adel was very friendly, even the day of my arr...","adel was very friendly, even the day of my arr...",0.318611
4,21853,22871843,2014-11-16,4072731,Nati,Abdel ha sido un magnífico anfitrión y la mejo...,es,Abdel has been a magnificent host and the best...,abdel has been a magnificent host and the best...,0.556667
...,...,...,...,...,...,...,...,...,...,...
1205941,1364524253446459367,1370233593987416123,2025-03-05,95293499,Paula,Me ha encantado. El piso está precioso muy nue...,es,I loved it. The floor is very new and the loca...,i loved it. the floor is very new and the loca...,0.444318
1205942,1364616759503109256,1368127111176342942,2025-03-02,368146996,Clémentine,"Très bon séjour chez Juan, le logement était p...",fr,"Very good stay with Juan, the accommodation wa...","very good stay with juan, the accommodation wa...",0.548571
1205943,1365007887657848515,1370244234430465527,2025-03-05,283312107,Hernan Enrique,Recomendable para una persona que quiere hacer...,es,Recommended for a person who wants to do a sho...,recommended for a person who wants to do a sho...,0.068750
1205944,1365007887657848515,1373818429800403177,2025-03-10,184521825,Carlos,"Es una Host súper amable, fue atenta en todo m...",es,"It is a super friendly host, it was attentive ...","it is a super friendly host, it was attentive ...",0.421667


In [ ]:
df_textblob = df_textblob[df_textblob['Sentiment_Score_Textblob'] != -90]

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x=df_textblob["Sentiment_Score_Textblob"])

plt.xlabel("Sentiment Score (-1 to 1)")
plt.title("Boxplot de Textblob")

plt.show()

In [ ]:
media_sentimiento_textblob = df_textblob.groupby('listing_id', as_index=False)['Sentiment_Score_Textblob'].mean()

In [ ]:
media_sentimiento_textblob

,listing_id,Sentiment_Score_Textblob
0,21853,0.407243
1,30320,0.372723
2,30959,0.395573
3,40916,0.439657
4,62423,0.451647
...,...,...
20053,1365007887657848515,0.245208
20054,1366534590465861868,0.643750
20055,1366853173044267909,0.700000
20056,1368304377290123314,0.243750


In [ ]:
df_merged = media_sentimiento_VADER.merge(media_sentimiento_textblob, left_on='listing_id', right_on='listing_id', how='left')

In [ ]:
df_merged

,listing_id,Sentiment_Score_VADER,Sentiment_Score_Textblob
0,21853,0.872903,0.407243
1,30320,0.824868,0.372723
2,30959,0.522625,0.395573
3,40916,0.784288,0.439657
4,62423,0.771982,0.451647
...,...,...,...
20053,1365007887657848515,0.556900,0.245208
20054,1366534590465861868,0.822100,0.643750
20055,1366853173044267909,0.440400,0.700000
20056,1368304377290123314,0.756900,0.243750


####**AFINN**

In [ ]:
def preprocess_for_afinn(text):
    if isinstance(text, str):
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)
        text = re.sub(r"\d+", "", text)
        text = text.strip()
    return text # no quitamos puntuación ni pasamos a minúsculas

In [ ]:
df_afinn=pd.read_csv('translated_reviews.csv')

In [ ]:
df_afinn=df_afinn.dropna(subset=['translated_review'])

In [ ]:
df_afinn['clean_review']=df_afinn['translated_review'].apply(preprocess_for_afinn)

In [ ]:
afinn = Afinn()

In [ ]:
df_afinn['Sentiment_Score_Afinn']=df_afinn['clean_review'].apply(lambda x: afinn.score(x) if pd.notnull(x) else None)

In [ ]:
df_afinn

,listing_id,id,date,reviewer_id,reviewer_name,comments,language,translated_review,clean_review,Sentiment_Score_Afinn
0,21853,21051116,2014-10-10,8506071,Pedro Abel,"Mi experiencia en casa de Adel fue buena, aunq...",es,"My experience in Adel's house was good, althou...","My experience in Adel's house was good, althou...",6.0
1,21853,21268157,2014-10-13,19062863,Olga,Adel is a very warm person and thoughtful! He...,en,Adel is a very warm person and thoughtful! He...,Adel is a very warm person and thoughtful! He ...,11.0
2,21853,22527396,2014-11-09,6099461,Hugh,Adel is an exceptionally welcoming host. I was...,en,Adel is an exceptionally welcoming host. I was...,Adel is an exceptionally welcoming host. I was...,10.0
3,21853,22684525,2014-11-11,23357657,Sendy,"Adel fue muy amable, incluso el día de mi lleg...",es,"Adel was very friendly, even the day of my arr...","Adel was very friendly, even the day of my arr...",11.0
4,21853,22871843,2014-11-16,4072731,Nati,Abdel ha sido un magnífico anfitrión y la mejo...,es,Abdel has been a magnificent host and the best...,Abdel has been a magnificent host and the best...,15.0
...,...,...,...,...,...,...,...,...,...,...
1205941,1364524253446459367,1370233593987416123,2025-03-05,95293499,Paula,Me ha encantado. El piso está precioso muy nue...,es,I loved it. The floor is very new and the loca...,I loved it. The floor is very new and the loca...,2.0
1205942,1364616759503109256,1368127111176342942,2025-03-02,368146996,Clémentine,"Très bon séjour chez Juan, le logement était p...",fr,"Very good stay with Juan, the accommodation wa...","Very good stay with Juan, the accommodation wa...",14.0
1205943,1365007887657848515,1370244234430465527,2025-03-05,283312107,Hernan Enrique,Recomendable para una persona que quiere hacer...,es,Recommended for a person who wants to do a sho...,Recommended for a person who wants to do a sho...,2.0
1205944,1365007887657848515,1373818429800403177,2025-03-10,184521825,Carlos,"Es una Host súper amable, fue atenta en todo m...",es,"It is a super friendly host, it was attentive ...","It is a super friendly host, it was attentive ...",8.0


In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x=df_afinn["Sentiment_Score_Afinn"])
plt.xlabel("Sentiment Score (-1 to 1)")
plt.title("Boxplot de Afinn")

plt.show()

In [ ]:
media_sentimiento_Afinn = df_afinn.groupby('listing_id', as_index=False)['Sentiment_Score_Afinn'].mean()

In [ ]:
media_sentimiento_Afinn

,listing_id,Sentiment_Score_Afinn
0,21853,10.030303
1,30320,9.859649
2,30959,4.875000
3,40916,7.571429
4,62423,8.805430
...,...,...
20053,1365007887657848515,5.000000
20054,1366534590465861868,5.000000
20055,1366853173044267909,3.000000
20056,1368304377290123314,2.000000


In [ ]:
media_sentimiento_Afinn.to_csv('afinn_todos.csv')

In [ ]:
df_merged = df_merged.merge(media_sentimiento_Afinn, left_on='listing_id', right_on='listing_id', how='left')

In [ ]:
df_merged

,listing_id,Sentiment_Score_VADER,Sentiment_Score_Textblob,Sentiment_Score_Afinn
0,21853,0.872903,0.407243,10.030303
1,30320,0.824868,0.372723,9.859649
2,30959,0.522625,0.395573,4.875000
3,40916,0.784288,0.439657,7.571429
4,62423,0.771982,0.451647,8.805430
...,...,...,...,...
20053,1365007887657848515,0.556900,0.245208,5.000000
20054,1366534590465861868,0.822100,0.643750,5.000000
20055,1366853173044267909,0.440400,0.700000,3.000000
20056,1368304377290123314,0.756900,0.243750,2.000000


In [ ]:
df_merged.to_csv('reviews_sentiment_analysis.csv')